In [2]:
import sys

# #only do this once
# sys.path.append("/home/heineike/github/expression_broad_data")
# print(sys.path)

#Indicate operating environment and import core modules
#import os
# location_input = input("what computer are you on? a = Bens, b = gpucluster, c = other   ")
# location_dict = {'a': "C:\\Users\Ben\Documents\GitHub\expression_broad_data", 'b': "/home/heineike/github/expression_broad_data/",'c':'you need to add your location to the location_dict'}
# data_processing_dir = location_dict[location_input]
# print("base directory is " + base_dir)
# os.chdir(base_dir) 
print("I am about to import a library")
from core import expression_plots 
from core import io_library 

['', '/usr/lib/python35.zip', '/usr/lib/python3.5', '/usr/lib/python3.5/plat-x86_64-linux-gnu', '/usr/lib/python3.5/lib-dynload', '/usr/local/lib/python3.5/dist-packages', '/usr/local/lib/python3.5/dist-packages/cmonkey2-1.2.6-py3.5.egg', '/usr/local/lib/python3.5/dist-packages/SQLAlchemy_Utils-0.32.14-py3.5.egg', '/usr/local/lib/python3.5/dist-packages/svgwrite-1.1.11-py3.5.egg', '/usr/local/lib/python3.5/dist-packages/Routes-2.4.1-py3.5.egg', '/usr/local/lib/python3.5/dist-packages/CherryPy-10.2.2-py3.5.egg', '/usr/local/lib/python3.5/dist-packages/biopython-1.69-py3.5-linux-x86_64.egg', '/usr/local/lib/python3.5/dist-packages/configparser-3.5.0-py3.5.egg', '/usr/local/lib/python3.5/dist-packages/rpy2-2.8.6-py3.5-linux-x86_64.egg', '/usr/local/lib/python3.5/dist-packages/repoze.lru-0.6-py3.5.egg', '/usr/local/lib/python3.5/dist-packages/portend-1.8-py3.5.egg', '/usr/local/lib/python3.5/dist-packages/cheroot-5.5.0-py3.5.egg', '/usr/local/lib/python3.5/dist-packages/tempora-1.7-py3.5.e

In [8]:
import os
base_dir = '/home/heineike/rna_seq_data/20170817_kl_sc_pka/'
data_processing_dir = '/home/heineike/github/expression_broad_data/expression_data/'
#os.chdir(base_dir) 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style="ticks", color_codes=True)
import pickle
import plotly.plotly as py
py.sign_in('heineike02', 'APPjKrtARaN2ZgUYIkqr')
import subprocess
from collections import Counter
import scipy.stats as stats


In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib 
plt.ioff()
#%matplotlib inline
#%matplotlib notebook

Using matplotlib backend: TkAgg


In [3]:
#Load sample metadata table
metadata = pd.read_table('/home/heineike/rna_seq_data/20170817_kl_sc_pka/metadata/20170817_klscpka_metadata.txt')
metadata.set_index('Label', inplace=True)

#extract alignment quality control metadata: 

qc_items = ["Number of input reads",
                "Uniquely mapped reads %",
                "% of reads mapped to multiple loci",
                "% of reads unmapped: too many mismatches",
                "% of reads unmapped: too short",
                "% of reads unmapped: other",
                "% of chimeric reads"]

for new_column_name in qc_items: 
    metadata[new_column_name]=np.nan

for label in metadata.index:
    fname = base_dir +  str(label) + "_Log.final.out"
    with open(fname,'r') as logfile:
        for line in logfile: 
            qc_stat = line.split("|")[0].strip() 
            if qc_stat in set(qc_items):
                metadata.loc[label,qc_stat]=line.split("|")[1].strip()
            




In [4]:
metadata

,Name,Code,Number of input reads,Uniquely mapped reads %,% of reads mapped to multiple loci,% of reads unmapped: too many mismatches,% of reads unmapped: too short,% of reads unmapped: other,% of chimeric reads
Label,,,,,,,,,
7001,KL_AS_noNMPP1_SDC_t40_rep1,CAGCGT,2945897,82.24%,11.27%,0.00%,6.45%,0.00%,0.00%
7002,KL_AS_yesNMPP1_SDC_t40_rep1,GATCAC,2687210,82.20%,12.83%,0.00%,4.91%,0.00%,0.00%
7003,KL_AS_noNMPP1_SDC_t20_rep2,ACCAGT,3506771,79.91%,13.53%,0.00%,6.50%,0.00%,0.00%
7005,KL_AS_noNMPP1_SDC_t40_rep2,ACATTA,3108677,83.05%,10.75%,0.00%,6.13%,0.00%,0.00%
7006,KL_AS_yesNMPP1_SDC_t40_rep2,GTGTAG,3431627,86.94%,9.32%,0.00%,3.68%,0.00%,0.00%
7007,SC_AS_noNMPP1_SDC_t20_rep2,CTAGTC,3224759,88.57%,9.62%,0.00%,1.18%,0.06%,0.00%
7008,SC_AS_yesNMPP1_SDC_t20_rep2,TGTGCA,3218378,83.90%,12.89%,0.00%,2.37%,0.06%,0.00%
7009,SC_AS_noNMPP1_SDC_t40_rep2,TCAGGA,3358827,76.40%,16.21%,0.00%,6.29%,0.07%,0.00%
7010,SC_AS_yesNMPP1_SDC_t40_rep2,CGGTTA,3547868,87.13%,10.32%,0.00%,1.69%,0.07%,0.00%


7014, 7045, and 7046 fail because they have too many reads that are too short to be unmapped. 

In [5]:
#Remove rows that fail QC (decided to do this after looking af full histogram/correlation plot below)
metadata.drop([7014,7045,7046],inplace = True)
metadata


,Name,Code,Number of input reads,Uniquely mapped reads %,% of reads mapped to multiple loci,% of reads unmapped: too many mismatches,% of reads unmapped: too short,% of reads unmapped: other,% of chimeric reads
Label,,,,,,,,,
7001,KL_AS_noNMPP1_SDC_t40_rep1,CAGCGT,2945897,82.24%,11.27%,0.00%,6.45%,0.00%,0.00%
7002,KL_AS_yesNMPP1_SDC_t40_rep1,GATCAC,2687210,82.20%,12.83%,0.00%,4.91%,0.00%,0.00%
7003,KL_AS_noNMPP1_SDC_t20_rep2,ACCAGT,3506771,79.91%,13.53%,0.00%,6.50%,0.00%,0.00%
7005,KL_AS_noNMPP1_SDC_t40_rep2,ACATTA,3108677,83.05%,10.75%,0.00%,6.13%,0.00%,0.00%
7006,KL_AS_yesNMPP1_SDC_t40_rep2,GTGTAG,3431627,86.94%,9.32%,0.00%,3.68%,0.00%,0.00%
7007,SC_AS_noNMPP1_SDC_t20_rep2,CTAGTC,3224759,88.57%,9.62%,0.00%,1.18%,0.06%,0.00%
7008,SC_AS_yesNMPP1_SDC_t20_rep2,TGTGCA,3218378,83.90%,12.89%,0.00%,2.37%,0.06%,0.00%
7009,SC_AS_noNMPP1_SDC_t40_rep2,TCAGGA,3358827,76.40%,16.21%,0.00%,6.29%,0.07%,0.00%
7010,SC_AS_yesNMPP1_SDC_t40_rep2,CGGTTA,3547868,87.13%,10.32%,0.00%,1.69%,0.07%,0.00%


In [6]:
#Add in columns for species, strain, NMPP1, Media, Time, and Replicate write metadata file suitable for DESEQ

exp_metadata_names = ["species", 
               "strain",
               "NMPP1",
               "media",
               "time",
               "replicate"]

exp_metadata = {name: [] for name in exp_metadata_names}

for label in metadata.index:
    exp_metadata_vec = metadata.loc[label,"Name"].split('_')
    for ind, name in enumerate(exp_metadata_names):
        exp_metadata[name].append(exp_metadata_vec[ind])

#could potentially clean up NMPP1 value to be minus and plus but will stick with it as is for now. 

for name in exp_metadata_names: 
    metadata[name]=exp_metadata[name]

species = ['KL', 'SC']
for spec in species: 
    metadata_spec = metadata[metadata['species']==spec]
    deseq_metadata_spec = metadata_spec.loc[:,['strain','NMPP1','media','time']]
    deseq_metadata_spec.to_csv(path_or_buf=data_processing_dir + '20170817_klscpka/20170817_klscpka_metadata_deseq_' + spec + '.csv')
#                       ['AS1_minus', 'AS','minus'],
#                       ['AS1_plus', 'AS', 'plus'],
#                       ['AS2_minus', 'AS','minus'],
#                       ['AS2_plus', 'AS', 'plus'],
#                       ['WT_minus', 'WT', 'minus'],
#                       ['WT_plus', 'WT', 'plus']]

# with open(base_dir + "\expression_data\kl_PKA_as_20160824\kl_NMPP1_metadata.csv", "w") as f:
#     writer = csv.writer(f)
#     writer.writerows(kl_PKA_metadata)
    
# sc_PKA_metadata = [['sample_name', 'strain','NMPP1'],
#                       ['AS_plus','AS', 'plus'],
#                       ['WT_minus','WT', 'minus'],
#                       ['WT_plus','WT', 'plus']]

# with open(base_dir + "\expression_data\SCer_NMPP1_RNA_Seq\sc_NMPP1_metadata.csv", "w") as f:
#     writer = csv.writer(f)
#     writer.writerows(sc_PKA_metadata)


In [7]:
deseq_metadata_spec

,strain,NMPP1,media,time
Label,,,,
7007,AS,noNMPP1,SDC,t20
7008,AS,yesNMPP1,SDC,t20
7009,AS,noNMPP1,SDC,t40
7010,AS,yesNMPP1,SDC,t40
7011,AS,noNMPP1,SDC,t20
7012,AS,yesNMPP1,SDC,t20
7013,WT,noNMPP1,SDC,t40
7017,AS,noNMPP1,SDC,t20
7018,AS,yesNMPP1,SDC,t20


In [11]:
#Load count data for all samples

#Ran STAR on the GPU cluster /home/heineike/github/rna_seq_processing/star/20170817_klscpka_alignment.py

#make a data panel containing counts for each species. 
#Keep only the second column - this is stranded data so this corresponds to the strand that is listed in the GFF file. 

species = ['KL', 'SC']
data_panels = {}
counts_raw = {}
counts = {}
rows_to_remove = ['N_unmapped','N_multimapping','N_noFeature','N_ambiguous']


for spec in species: 
    labels = metadata[metadata['species']==spec].index
    
    data_dict = {}
    for label in labels:
        data_dict[label] = pd.read_table(base_dir + str(label) + '_ReadsPerGene.out.tab', index_col = 0, header = None)

    data_panels[spec] = pd.Panel(data_dict)
    
    #take the second column which is for stranded library preparation protocols
    counts_raw_w_metadata = data_panels[spec].minor_xs(2)

    #remove the metadata rows.
    counts_raw[spec] = counts_raw_w_metadata.drop(rows_to_remove)

    ### Do I need to do any normalization?  Why is this the right kind of normalization? 

    #add up the counts and normalize by total - then multiplies by the mean number of counts to get a number on the same scale 
    #as origina counts.  
    counts[spec] = counts_raw[spec].copy()
    total_reads = counts_raw[spec].sum()
    for label in labels: 
        norm_reads = (counts_raw[spec][label]*np.mean(total_reads))/total_reads[label]
        counts[spec][str(label) + '_norm'] = norm_reads


# Save raw and normalized counts as an entire data structure
fname_counts_pkl = data_processing_dir + '20170817_klscpka/20170817_klscpka_counts.pkl'
output = open(fname_counts_pkl, 'wb')
pickle.dump(counts, output)
output.close()

In [12]:
#make CSV for use in DESEQ calculations
for spec in species:
    counts_raw[spec].to_csv(path_or_buf=data_processing_dir + '20170817_klscpka/20170817_klscpka_counts_' + spec + '.txt', sep="\t")

In [13]:
#Scatterplots of log normalized plots 

#This error didn't seem to occur for nbagg background hmm
psuedocount = 1.0
for spec in species: 
    labels = metadata[metadata['species']==spec].index
    norm_labels = [str(label)+'_norm' for label in labels]
    normcounts_KL = counts[spec].loc[:,norm_labels]  
    normcounts_KL = normcounts_KL + 1
    normcounts_KL = normcounts_KL.apply(np.log10)

    corrplot = sns.pairplot(normcounts_KL)
    fig = corrplot.fig
    fig.savefig(base_dir + 'plots/' + spec + 'corrplot.png') 
    plt.close(fig)

TclError: no display name and no $DISPLAY environment variable

In [ ]:
#correlation matrix - raw counts 

#incorporate S.Cer data (solis) and older K.Lac data

In [ ]:


#This data and the raw data was then converted to rlog data and DEseq data in R using the file
# deseq2_analysis_KL_PKA.nb
#That file contains Quality Control Plots of the data. 

In [39]:
pmNmpp1_SDC_t40_lfc["KL"]

,orf_name,PKA(AS)+1NMPP1,padj
0,KLLA0D06941g,-5.686987,5.512722e-66
1,KLLA0E16171g,-6.216423,5.120522e-64
2,KLLA0F23243g,-5.072907,8.160543e-54
3,KLLA0B02937g,-5.019739,2.187869e-51
4,KLLA0F06831g,-4.932962,9.004291e-50
5,KLLA0F08657g,-5.366840,2.499317e-47
6,KLLA0D16027g,-5.146560,4.410324e-47
7,KLLA0E12453g,-4.797737,5.494793e-47
8,KLLA0C13519g,-5.232692,5.494793e-47
9,KLLA0D03410g,-4.817955,1.183183e-46


In [43]:
#Import DE Seq Data, [later rlog and raw count data]
pmNmpp1_SDC_t40_lfc = {}
deseq_pmNmpp1_SDC_t40 = {} 

for spec in species:
    deseq_pmNmpp1_SDC_t40[spec] = pd.read_csv(data_processing_dir + "20170817_klscpka/20170817_klscpka_DESEQ_SDC_t40_AsMinvAsPlus_" + spec + ".csv")
    

    #Convert kl_genename to standard format. 
    if spec == 'KL':
        rename_dict = {deseq_pmNmpp1_SDC_t40[spec].columns[0]:'kl_genename_GFF'}
        deseq_pmNmpp1_SDC_t40[spec].rename(columns = rename_dict, inplace = True)
        kl_genename = []
        for gene in deseq_pmNmpp1_SDC_t40[spec]['kl_genename_GFF']:
            if gene[0:5]=='KLLA0':
                new_gene = gene.split('_')[0]+gene.split('_')[1]
            else: 
                new_gene = gene
            kl_genename.append(new_gene)
        deseq_pmNmpp1_SDC_t40[spec]['orf_name'] = kl_genename
    elif spec == 'SC': 
        rename_dict = {deseq_pmNmpp1_SDC_t40[spec].columns[0]:'orf_name'}
        deseq_pmNmpp1_SDC_t40[spec].rename(columns = rename_dict, inplace = True)

    #Make LFC dataframe which only has the log fold change and the padj

    pmNmpp1_SDC_t40_lfc[spec] = deseq_pmNmpp1_SDC_t40[spec].loc[:,['orf_name', 'log2FoldChange','padj']]
    pmNmpp1_SDC_t40_lfc[spec].rename(index=str, columns={"log2FoldChange": "PKA(AS)+1NMPP1"}, inplace = True)

# Save raw and normalized counts as an entire data structure
with open(data_processing_dir + "20170817_klscpka/LFC_SDC_t40_AsMinvAsPlus.pkl", 'wb') as pkl_out:
    pickle.dump(pmNmpp1_SDC_t40_lfc, pkl_out)

In [5]:
#load LFC data for SDC t40
with open(data_processing_dir + "20170817_klscpka/LFC_SDC_t40_AsMinvAsPlus.pkl", "rb") as pkl_in:
    pmNmpp1_SDC_t40_lfc = pickle.load(pkl_in)

In [6]:
#Load KL to SC ortholog mapping

kl_orthologs = pd.read_pickle(data_processing_dir + "ortholog_files_YGOB/kl_orthologs.pkl")


In [7]:
#Map KL PKA inhibition data onto related SC orthologs
#First change column name for kl_nmpp1_lfc data to match ortholog table for merge. 
kl_nmpp1_lfc_renamed = pmNmpp1_SDC_t40_lfc['KL'].copy()
kl_nmpp1_lfc_renamed.columns = ['kl_genename'] + list(pmNmpp1_SDC_t40_lfc['KL'].columns[1:])
kl_PKA_data_sc_orthologs = pd.merge(kl_orthologs, kl_nmpp1_lfc_renamed, on = 'kl_genename', how = 'inner')

#add in SC_common_name and SC_common name
sc_nmpp1_lfc_renamed = pmNmpp1_SDC_t40_lfc['SC'].copy()
sc_nmpp1_lfc_renamed.columns = ['sc_genename'] + list(pmNmpp1_SDC_t40_lfc['SC'].columns[1:])
sc_nmpp1_lfc_renamed.set_index('sc_genename',inplace = True)
sc_genenames = sc_nmpp1_lfc_renamed.index
sc_commonnames = io_library.SC_common_name_lookup(sc_genenames)
sc_nmpp1_lfc_renamed['SC_common_name']= sc_commonnames
#Just noticed that IMP2 has a common name IMP2' - should troubleshoot that someday. 

kl_sc_PKA_data = kl_PKA_data_sc_orthologs.join(sc_nmpp1_lfc_renamed, on = 'sc_genename',lsuffix = '_KL',rsuffix = '_SC')


In [10]:
kl_sc_PKA_data.to_pickle(data_processing_dir + "20170817_klscpka/kl_sc_PKA_data_SDC_t40_AsMinvAsPlus.pkl")

In [4]:
kl_sc_PKA_data = pd.read_pickle(data_processing_dir + "20170817_klscpka/kl_sc_PKA_data_SDC_t40_AsMinvAsPlus.pkl")

In [179]:
#Lookup by SC common name
#Lookup by SC common name
jso_msn2_targets = ['SSA1', 'SSA4', 'HSP104', 'CTT1', 'TPK1', 'TFS1', 'DDR2', 'TSA1', 'HSP12', 'TMA10', 'TPS2', 'RTC3', 'GPH1', 'HXK1', 'TSA2', 'CYC7', 'PNC1', 'GSY1', 'FMP48', 'SS13', 'ALD4', 'YNR014W', 'HOR2', 'YOR173W', 'RAS2', 'GLK1', 'GPD1', 'HSP26', 'SIP18', 'TKL2', 'FMP16', 'ALD3', 'PGM2']
mitochondrial_translation = ['MRPL16','PET112','VAR1','MRP21','MRPL36','MRPS9']
common_name_query = ['HUL4','UBC5','UBC8','UBI4','UBP15','BMH2','ARC18','HSP42','YAK1','BCY1','PDE1','PDE2','TPK1','TPK2','TPK3','TOR1','CIT1','ZWF1']
#['TPK3','TPK2','TPK1','MIG1','MIG2','MIG3','CAT8','HAP2','HAP4','HAP5','SIP2','TUP1','CYC8']#['VPS8','PEP1','VPS15','VID24','CCZ1','ARL1','APM3'] #protein targeting to vacuole #['FMP45','SWF1','SUR7','YNL194C','SCD5']#['CAR1','CAR2']#['DEP1', 'FYV10','GID7','GID8','IGD1','NUP2','PCL10','PCL8','PHO85','PSK2','RMD5','UBC8','UBP14','UME6','VID24','VID28','VID30','YTR066C']# ['CTT1']  #['SSA1', 'SSA2','AHA1','BTN2', 'CPR6','CUR1','FES1','HCH1','HSC82','HSP42','HSP104','HSP78','HSP82','MBF1','MDJ1','SIS1','STI1','YDJ1']
kl_sc_PKA_data[kl_sc_PKA_data['SC_common_name'].isin(common_name_query)].sort_values(by='PKA(AS)+1NMPP1_KL')


,N SCer Orthologs,kl_genename,sc_genename,PKA(AS)+1NMPP1_KL,padj_KL,PKA(AS)+1NMPP1_SC,padj_SC,SC_common_name
1474,1,KLLA0A03619g,YOR360C,-1.912907,7.528617e-04,-1.415506,4.618725e-02,PDE2
3900,1,KLLA0D11528g,YGL248W,-0.223134,8.432386e-01,1.431112,6.322347e-05,PDE1
4739,2,KLLA0E12563g,YDR059C,0.191731,7.398819e-01,2.105107,2.595713e-09,UBC5
4380,2,KLLA0B13948g,YJR066W,0.615947,1.601963e-01,1.050073,3.231961e-04,TOR1
901,1,KLLA0D19855g,YNL241C,0.652318,5.977392e-01,1.610259,1.294243e-03,ZWF1
4942,2,KLLA0F12100g,YDR099W,0.894887,3.437978e-01,1.506584,5.224099e-05,BMH2
4365,2,KLLA0F12760g,YNR001C,1.012124,3.537483e-01,2.682549,1.291298e-11,CIT1
262,1,KLLA0B07821g,YLR370C,1.241166,9.193630e-03,1.282490,9.580574e-04,ARC18
2260,1,KLLA0A11198g,YEL012W,1.402026,1.436166e-03,3.638090,7.527784e-14,UBC8
4456,2,KLLA0B07205g,YJL164C,1.534305,1.170214e-03,3.260174,8.858192e-23,TPK1


In [81]:
test = set([0,2,7,11,12,14])
set(kl_sc_PKA_data_sig.index)&test
kl_sc_PKA_data.loc[list(set(kl_sc_PKA_data_sig.index)&test)]

,N SCer Orthologs,kl_genename,sc_genename,PKA(AS)+1NMPP1_KL,padj_KL,PKA(AS)+1NMPP1_SC,padj_SC,SC_common_name
0,1,KLLA0E08273g,YIL093C,-3.214403,1.019397e-07,-0.188637,7.437343e-01,RSM25
2,1,KLLA0F12056g,YKR076W,4.035633,3.148716e-25,3.899617,2.145520e-32,ECM4
7,1,KLLA0F04499g,YNL067W,-4.451254,7.609392e-22,-1.909883,1.313254e-09,RPL9B


In [89]:
len(kl_sc_PKA_data)

4976

dict_keys(['Sig KL and SC', 'Sig KL only', 'Sig SC only', 'Unsig KL and SC'])

In [5]:
#Highlight hits that are statistically significant for K.Lactis 

#Add column which highlights only pvalues below a threshold

#should eventually have a principled way of picking these
pthreshold_KL = 10e-5
pthreshold_SC = 10e-9

# stre_eq1 = [kl_gene for kl_gene, n in jso_prom_hits_kl_hitcount.items() if n==1]
# stre_gt1 = [kl_gene for kl_gene, n in jso_prom_hits_kl_hitcount.items() if n>1]

kl_sc_PKA_data_klsig = kl_sc_PKA_data[kl_sc_PKA_data['padj_KL'] < pthreshold_KL]
kl_sc_PKA_data_scsig = kl_sc_PKA_data[kl_sc_PKA_data['padj_SC'] < pthreshold_SC]

klscsig_ind = set(kl_sc_PKA_data_klsig.index)&set(kl_sc_PKA_data_scsig.index)
klsig_scunsig_ind = set(kl_sc_PKA_data_klsig.index)-klscsig_ind
scsig_klunsig_ind = set(kl_sc_PKA_data_scsig.index)-klscsig_ind
unsig_ind = set(kl_sc_PKA_data.index)-(set(kl_sc_PKA_data_klsig.index)|set(kl_sc_PKA_data_scsig.index))

print([len(klscsig_ind),len(klsig_scunsig_ind),len(scsig_klunsig_ind),len(unsig_ind)])

kl_sc_PKA_data_klsig_scunsig = kl_sc_PKA_data.loc[list(klsig_scunsig_ind)]
kl_sc_PKA_data_scsig_klunsig = kl_sc_PKA_data.loc[list(scsig_klunsig_ind)]
kl_sc_PKA_data_klscsig = kl_sc_PKA_data.loc[list(klscsig_ind)]
kl_sc_PKA_data_unsig = kl_sc_PKA_data.loc[list(unsig_ind)]

# print([len(kl_sc_PKA_data_klscsig),len(kl_sc_PKA_data_klsig_scunsig),len(kl_sc_PKA_data_scsig_klunsig),len(kl_sc_PKA_data_unsig)])





[462, 486, 319, 3709]


In [6]:
" ".join((kl_sc_PKA_data_klsig_scunsig[kl_sc_PKA_data_klsig_scunsig['PKA(AS)+1NMPP1_KL']>0]['sc_genename']))
#" ".join((kl_sc_PKA_data_klscsig[kl_sc_PKA_data_klscsig['PKA(AS)+1NMPP1_KL']>0]['sc_genename']))  #Need to fix this - keep only SC greater than 0 too
#" ".join((kl_sc_PKA_data_scsig_klunsig[kl_sc_PKA_data_scsig_klunsig['PKA(AS)+1NMPP1_SC']>0]['sc_genename']))

'YDR036C YOR003W YFR021W YPR061C YDL139C YJL103C YHR067W YCR010C YGL081W YML020W YDR425W YOR040W YGL126W YIL045W YIL057C YLR109W YPR122W YMR160W YLR299W YLL042C YMR114C YOR208W YDR326C YHR080C YLR219W YDR461C-A YML051W YFL026W YCR076C YHR163W YIL099W YDL222C YNL165W YHR044C YKR089C YOR081C YLR040C YOL024W YPR006C YIR029W YBL016W YOR052C YDR305C YDR446W YIL156W YBR203W YKR098C YLR456W YDR294C YIL146C YPL133C YLR240W YLR189C YPL070W YDR068W YDR387C YDR122W YLR096W YFL030W YNL251C YJL157C YGR142W YNL275W YMR232W YPR158W Scer_YGOB_SDC25 YLR310C YER026C YPL071C YMR156C YAL051W YOR363C YEL047C YKR067W YJR051W YCR036W YMR226C YKL213C YMR008C YOL011W YOL129W YLR423C YLR211C YCR073W-A YNR034W YER037W YGL224C YDR525W-A YHR004C YPL048W YNR007C YOR059C YKR009C YBR273C YJL048C YDR259C YOR028C YDR207C YMR119W YNL008C YPR002W YJR111C YER132C YGL197W YOL075C YNL086W YGR049W YIL152W YCR068W YLR173W YBR218C YGL062W YOR386W YIL037C YER015W YJL036W YIL002C YHR150W YDL015C YDR480W YPL049C YMR302C YKL127W Y

In [14]:
oddsratio, pvalue = stats.fisher_exact([[28, 231-28], [296-28, 6433-231]],alternative = 'greater')

In [15]:
pvalue

9.0139542963655987e-07

In [ ]:
#plot figure in plot.ly
condition_set = {'Unsig KL and SC' : kl_sc_PKA_data_unsig, 'Sig KL only': kl_sc_PKA_data_klsig_scunsig, 'Sig SC only': kl_sc_PKA_data_scsig_klunsig, 'Sig KL and SC': kl_sc_PKA_data_klscsig}

fig_data = [
  		{
  			'x': condition_set[condition]['PKA(AS)+1NMPP1_KL'], 
        	'y': condition_set[condition]['PKA(AS)+1NMPP1_SC'], 
        	'text': condition_set[condition]['SC_common_name'], 
        	'mode': 'markers', 
        	'name': condition} for condition in condition_set.keys()]
fig = {
    'data': fig_data,
    'layout': {
        'xaxis': {'title': 'KL LFC 4uM NMPP1'},
        'yaxis': {'title': 'SC LFC 4uM NMPP1'}
    }
}

# IPython notebook
# py.iplot(fig, filename='pandas/multiple-scatter')

url = py.plot(fig, filename='sc_kl_scatter_sig_highlight_20170821')

In [97]:
#Get a list of promoters and check for enrichment using meme

## build subset for JSO Msn2 targets
# jso_msn2_targets = ['SSA1', 'SSA4', 'HSP104', 'CTT1', 'TPK1', 'TFS1', 'DDR2', 'TSA1', 'HSP12', 'TMA10', 'TPS2', 'RTC3', 'GPH1', 'HXK1', 'TSA2', 'CYC7', 'PNC1', 'GSY1', 'FMP48', 'SS13', 'ALD4', 'YNR014W', 'HOR2', 'YOR173W', 'RAS2', 'GLK1', 'GPD1', 'HSP26', 'SIP18', 'TKL2', 'FMP16', 'ALD3', 'PGM2']
# jso_msn2_targets_kl_orth = list(kl_sc_PKA_data[kl_sc_PKA_data['SC_common_name'].isin(jso_msn2_targets)]['kl_genename'])

#Function that takes a species name, an input set of genenames, and an input control set of genenames and provides AME outut
#it also generates files containing the promoters at the same time. 


spec = 'SC'
#generates target and control gene lists from kl_sc_PKA_data.  Since there are 2 to 1 orthologs mapping from SC to KL KL lists
#need to have duplicates removed i.e. list(set(<gene_list>))
target_gene_list = list(set(kl_sc_PKA_data_klsig[kl_sc_PKA_data_klsig['PKA(AS)+1NMPP1_SC']>0.1]['sc_genename']))
control_gene_list = list(set(kl_sc_PKA_data['sc_genename']))
target_fname_prefix = 'sc_act_sig10e-5'
control_fname_prefix = 'sc_all_withorth'
motif = {'name': 'Jaspar_core_2016', 'fname': 'motifs/JASPAR_CORE_2016_fungi.meme' }

#optional inputs
promoter_dir = {'KL': data_processing_dir+'kl_promoters/' , 'SC': data_processing_dir + 'sc_promoters/'}
promoter_fname = {'KL': 'kl_promoters.pkl', 'SC': 'sc_promoters.pkl'}
ame_scoring = 'totalhits'
ame_method = 'fisher'
ame_pvalue_threshold = '0.05'


#extract promoters
promoters = pd.read_pickle(promoter_dir[spec] + promoter_fname[spec]) 
target_promoters = promoters.loc[target_gene_list,]
control_promoters = promoters.loc[control_gene_list,]  #for control using just promoters which have orthologs - but should I use all promoters? 


fname_prefixes = {'target':target_fname_prefix, 'control': control_fname_prefix}
promoter_lists = {'target':target_promoters, 'control': control_promoters}

for gene_set in ['target','control']:
    fname = promoter_dir[spec] + 'promoter_sets/' + fname_prefixes[gene_set] + '_promoters.fasta'
    with open(fname,'w') as f:
        for row in promoter_lists[gene_set].itertuples():
            if isinstance(row.prom_seq,str):
                header_line = '>' + row.Index + ' 700bp_upstream\n'
                seq_line = row.prom_seq + '\n'
                f.write(header_line)
                f.write(seq_line)
            else:
                print(row.Index + ' in ' + gene_set + ' set promoter value is not a string : ' + str(row.prom_seq)+ '. Skipping for promoter file.')


#Use subprocess to run meme commands: 

#ame --verbose 1 --oc . --control all_kl_promoters.fasta --bgformat 1 --scoring avg --method ranksum --pvalue-report-threshold 0.05 mito_promoters_kl.fasta db/JASPAR/JASPAR_CORE_2016_fungi.meme
motif_db = promoter_dir[spec]+ motif['fname']
target_sequences = promoter_dir[spec] + 'promoter_sets/' + fname_prefixes['target'] + '_promoters.fasta'
control_sequences = promoter_dir[spec] + 'promoter_sets/' + fname_prefixes['control'] + '_promoters.fasta'
output_dir = promoter_dir[spec] + 'ame_output'
file_prefix = target_fname_prefix + '_vs_' + control_fname_prefix + '_motif_' + motif['name'] + '_pVal_' + ame_pvalue_threshold


ame_command = [ "/home/kieran/meme/bin/ame",
              "--verbose", "2",
              "--oc", output_dir,
              "--control", control_sequences,
              "--bgformat", "1", 
              "--scoring", ame_scoring,
              "--method", ame_method, 
              "--pvalue-report-threshold", ame_pvalue_threshold, 
              target_sequences,
              motif_db]

ame_output = subprocess.run(ame_command,stdout = subprocess.PIPE) 

print(ame_output.returncode)

#change file prefix
for fname in ["ame.txt","ame.html"]:
    os.rename(output_dir + os.sep + fname, output_dir + os.sep + file_prefix + fname)


KLLA0D17666r in control set promoter value is not a string : nan. Skipping for promoter file.
KLLA0D17633r in control set promoter value is not a string : nan. Skipping for promoter file.
KLLA0D17677r in control set promoter value is not a string : nan. Skipping for promoter file.
KLLA0D17562r in control set promoter value is not a string : nan. Skipping for promoter file.
0


In [105]:
#generates target and control gene lists from kl_sc_PKA_data.  Since there are 2 to 1 orthologs mapping from SC to KL KL lists
#need to have duplicates removed i.e. list(set(<gene_list>))
spec = 'KL'
target_gene_list = list(set(kl_sc_PKA_data_klsig[kl_sc_PKA_data_klsig['PKA(AS)+1NMPP1_KL']>0.1]['kl_genename']))
control_gene_list = list(set(kl_sc_PKA_data['kl_genename']))
target_fname_prefix = 'kl_act_sig10e-5'
control_fname_prefix = 'kl_all_withorth'
motif = {'name': 'Jaspar_core_2016', 'fname': 'motifs/JASPAR_CORE_2016_fungi.meme' }

io_library.run_ame_analysis(spec, target_gene_list, control_gene_list,target_fname_prefix, control_fname_prefix, motif, 
                     promoter_dir = {'KL': data_processing_dir+'kl_promoters/' , 'SC': data_processing_dir + 'sc_promoters/'},
                     promoter_fname = {'KL': 'kl_promoters.pkl', 'SC': 'sc_promoters.pkl'},
                     ame_scoring = 'totalhits',
                     ame_method = 'fisher',
                     ame_pvalue_threshold = '0.04' )

KLLA0D17666r in control set promoter value is not a string : nan. Skipping for promoter file.
KLLA0D17633r in control set promoter value is not a string : nan. Skipping for promoter file.
KLLA0D17677r in control set promoter value is not a string : nan. Skipping for promoter file.
KLLA0D17562r in control set promoter value is not a string : nan. Skipping for promoter file.
ame output return code = 0


In [ ]:
print(len(kl_nmpp1_lfc_act_e5))

In [153]:
#KLac activated v.s. all promoter enrichment. 
with open(data_processing_dir + "20170817_klscpka/LFC_SDC_t40_AsMinvAsPlus.pkl", "rb") as pkl_in:
    pmNmpp1_SDC_t40_lfc = pickle.load(pkl_in)
kl_nmpp1_lfc_renamed = pmNmpp1_SDC_t40_lfc['KL'].copy()
kl_nmpp1_lfc_renamed.columns = ['kl_genename'] + list(pmNmpp1_SDC_t40_lfc['KL'].columns[1:])
kl_nmpp1_lfc_act_e5 = kl_nmpp1_lfc_renamed[(kl_nmpp1_lfc_renamed["PKA(AS)+1NMPP1"]>0).values & (kl_nmpp1_lfc_renamed["padj"]<10e-5).values]

target_gene_list = kl_nmpp1_lfc_act_e5['kl_genename']
control_gene_list = kl_nmpp1_lfc_renamed['kl_genename']
target_fname_prefix = 'kl_act_sig10e-5'
control_fname_prefix = 'kl_all'
motif = {'name': 'Jaspar_core_2016_avg_fisher', 'fname': 'motifs/JASPAR_CORE_2016_fungi.meme' }

io_library.run_ame_analysis(spec, target_gene_list, control_gene_list,target_fname_prefix, control_fname_prefix, motif, 
                     promoter_dir = {'KL': data_processing_dir+'kl_promoters/' , 'SC': data_processing_dir + 'sc_promoters/'},
                     promoter_fname = {'KL': 'kl_promoters.pkl', 'SC': 'sc_promoters.pkl'},
                     ame_scoring = 'avg',
                     ame_method = 'fisher',
                     ame_pvalue_threshold = '0.04' )

KLLA0C17886r in target set promoter value is not a string : nan. Skipping for promoter file.
KLLA0D08140r in target set promoter value is not a string : nan. Skipping for promoter file.
KLLA0C17886r in control set promoter value is not a string : nan. Skipping for promoter file.
KLLA0D11217r in control set promoter value is not a string : nan. Skipping for promoter file.
KLLA0D08140r in control set promoter value is not a string : nan. Skipping for promoter file.
KLLA0A02042r in control set promoter value is not a string : nan. Skipping for promoter file.
KLLA0C15752r in control set promoter value is not a string : nan. Skipping for promoter file.
KLLA0B07425r in control set promoter value is not a string : nan. Skipping for promoter file.
KLLA0B08118r in control set promoter value is not a string : nan. Skipping for promoter file.
EBG00000982146 in control set promoter value is not a string : nan. Skipping for promoter file.
EBG00000981872 in control set promoter value is not a string

ame output return code = 0


In [155]:
#Get enrichment for genes that are significant in Klac (activated) but not for SC v.s. all activated KL genes that have orthologs. 

spec = 'KL'
#generates target and control gene lists from kl_sc_PKA_data.  Since there are 2 to 1 orthologs mapping from SC to KL KL lists
#need to have duplicates removed i.e. list(set(<gene_list>))

target_gene_list = kl_sc_PKA_data_klsig_scunsig[kl_sc_PKA_data_klsig_scunsig['PKA(AS)+1NMPP1_KL']>0.0]['kl_genename']
control_gene_list = kl_sc_PKA_data_klsig[kl_sc_PKA_data_klsig['PKA(AS)+1NMPP1_KL']>0.0]['kl_genename']
target_fname_prefix = 'klactsig10e-5_scunsig10e-9'
control_fname_prefix = 'klactsig10e5'
motif = {'name': 'Jaspar_core_2016_avg', 'fname': 'motifs/JASPAR_CORE_2016_fungi.meme' }

io_library.run_ame_analysis(spec, target_gene_list, control_gene_list,target_fname_prefix, control_fname_prefix, motif, 
                     promoter_dir = {'KL': data_processing_dir+'kl_promoters/' , 'SC': data_processing_dir + 'sc_promoters/'},
                     promoter_fname = {'KL': 'kl_promoters.pkl', 'SC': 'sc_promoters.pkl'},
                     ame_scoring = 'avg',
                     ame_method = 'fisher',
                     ame_pvalue_threshold = '0.04' )

ame output return code = 0


In [149]:
#Get enrichment of RME1 for genes that are significant in Klac (activated) but not for SC v.s. all activated KL genes that have orthologs. 

spec = 'KL'
#generates target and control gene lists from kl_sc_PKA_data.  Since there are 2 to 1 orthologs mapping from SC to KL KL lists
#need to have duplicates removed i.e. list(set(<gene_list>))

target_gene_list = kl_sc_PKA_data_klsig_scunsig[kl_sc_PKA_data_klsig_scunsig['PKA(AS)+1NMPP1_KL']>0.0]['kl_genename']
control_gene_list = kl_sc_PKA_data_klsig[kl_sc_PKA_data_klsig['PKA(AS)+1NMPP1_KL']>0.0]['kl_genename']
target_fname_prefix = 'klactsig10e-5_scunsig10e-9'
control_fname_prefix = 'klactsig10e5'
motif = {'name': 'rme1', 'fname': 'motifs/rme1.meme' }
ame_pvalue_threshold = '0.05'

io_library.run_ame_analysis(spec, target_gene_list, control_gene_list,target_fname_prefix, control_fname_prefix, motif, 
                      ame_pvalue_threshold=ame_pvalue_threshold)

ame output return code = 0


In [144]:
# from core import io_library 

spec = 'KL'
#Maybe a repressor is introduced in K.Lactis genes that are not activated in S.Cer? 

#generates target and control gene lists from kl_sc_PKA_data.  Since there are 2 to 1 orthologs mapping from SC to KL KL lists
#need to have duplicates removed i.e. list(set(<gene_list>))

target_gene_list = kl_sc_PKA_data_scsig_klunsig[kl_sc_PKA_data_scsig_klunsig['PKA(AS)+1NMPP1_KL']>0.0]['kl_genename']
control_gene_list = kl_sc_PKA_data_scsig[kl_sc_PKA_data_scsig['PKA(AS)+1NMPP1_KL']>0.0]['kl_genename']
target_fname_prefix = 'scactsig10e9_klunsig10e5'
control_fname_prefix = 'scactsig10e9'
motif = {'name': 'Jaspar_core_2016', 'fname': 'motifs/JASPAR_CORE_2016_fungi.meme' }
ame_pvalue_threshold = '0.4' 

io_library.run_ame_analysis(spec, target_gene_list, control_gene_list,target_fname_prefix, control_fname_prefix, motif, 
                     ame_pvalue_threshold = ame_pvalue_threshold)

ame output return code = 0


In [146]:
# from core import io_library 

spec = 'SC'
#In compare pka inhibition active genes in SC versus all genes 

#generates target and control gene lists from kl_sc_PKA_data.  Since there are 2 to 1 orthologs mapping from SC to KL KL lists
#need to have duplicates removed i.e. list(set(<gene_list>))

with open(data_processing_dir + "20170817_klscpka/LFC_SDC_t40_AsMinvAsPlus.pkl", "rb") as pkl_in:
    pmNmpp1_SDC_t40_lfc = pickle.load(pkl_in)
sc_nmpp1_lfc_renamed = pmNmpp1_SDC_t40_lfc['SC'].copy()
sc_nmpp1_lfc_renamed.columns = ['sc_genename'] + list(pmNmpp1_SDC_t40_lfc['SC'].columns[1:])
sc_nmpp1_lfc_act_e5 = sc_nmpp1_lfc_renamed[(sc_nmpp1_lfc_renamed["PKA(AS)+1NMPP1"]>0).values & (sc_nmpp1_lfc_renamed["padj"]<10e-9).values]

target_gene_list = sc_nmpp1_lfc_act_e5['sc_genename']
control_gene_list = sc_nmpp1_lfc_renamed['sc_genename']
target_fname_prefix = 'sc_act_sig10e9'
control_fname_prefix = 'sc_all'
motif = {'name': 'Jaspar_core_2016', 'fname': 'motifs/JASPAR_CORE_2016_fungi.meme' }

io_library.run_ame_analysis(spec, target_gene_list, control_gene_list,target_fname_prefix, control_fname_prefix, motif, 
                     ame_pvalue_threshold = ame_pvalue_threshold)

YIL111W in target set promoter value is not a string : nan. Skipping for promoter file.
YIL111W in control set promoter value is not a string : nan. Skipping for promoter file.
YOR312C in control set promoter value is not a string : nan. Skipping for promoter file.
YMR242C in control set promoter value is not a string : nan. Skipping for promoter file.
Q0075 in control set promoter value is not a string : nan. Skipping for promoter file.
YJL041W in control set promoter value is not a string : nan. Skipping for promoter file.
YFL067W in control set promoter value is not a string : nan. Skipping for promoter file.
Q0297 in control set promoter value is not a string : nan. Skipping for promoter file.
YIL177W-A in control set promoter value is not a string : nan. Skipping for promoter file.
YFL057C in control set promoter value is not a string : nan. Skipping for promoter file.
YOL166W-A in control set promoter value is not a string : nan. Skipping for promoter file.
YML133W-B in control s

In [147]:
spec = 'SC'
#In SC compare genes activated for SC but not for KL with all genes activated for SC. 

target_gene_list = kl_sc_PKA_data_scsig_klunsig[kl_sc_PKA_data_scsig_klunsig['PKA(AS)+1NMPP1_SC']>0.0]['sc_genename']
control_gene_list = kl_sc_PKA_data_scsig[kl_sc_PKA_data_scsig['PKA(AS)+1NMPP1_SC']>0.0]['sc_genename']
target_fname_prefix = 'scactsig10e9_klunsig10e5'
control_fname_prefix = 'scactsig10e9'
motif = {'name': 'Jaspar_core_2016', 'fname': 'motifs/JASPAR_CORE_2016_fungi.meme' }

io_library.run_ame_analysis(spec, target_gene_list, control_gene_list,target_fname_prefix, control_fname_prefix, motif, 
                     ame_pvalue_threshold = ame_pvalue_threshold)

YIL111W in target set promoter value is not a string : nan. Skipping for promoter file.
YIL111W in control set promoter value is not a string : nan. Skipping for promoter file.
ame output return code = 0


In [20]:
#Build dataframe with motif hits for a number of transcription factors
motif_parameters_dict = {'msn2': ('20170224',{'kl':0.000710,'sc':0.000710}), 
                        'dot6': ('20170328',{'kl':1.0e-4,'sc':1.0e-4}),
                        'rap1':('20170328',{'kl':1.0e-4,'sc':1.0e-4}),
                        #'cat8':('20170328',{'kl':4.0e-4,'sc':4.0e-4}),
                        #'hac1':('20170328',{'kl':1.0e-4,'sc':1.0e-4}),
                        #'hsf1':('20170330',{'kl':1.0e-4,'sc':1.0e-4}),
                        #'oaf1':('20170330',{'kl':1.0e-4,'sc':1.0e-4}),
                        #'gsm1':('20170330',{'kl':1.0e-4,'sc':1.0e-4}),
                        #'hap3':('20170330',{'kl':1.0e-4,'sc':1.0e-4}),
                        #'AAGGGA':('20170330',{'kl':3.0e-4,'sc':3.0e-4},
                        #'azf1':('20170413',{'kl':3.0e-5,'sc':1.0e-5}), #more perfect matches with SC
                        #'crz1':('20170413',{'kl':1.0e-4,'sc':1.0e-4}),  #not many - low information
                        #'mig1':('20170413',{'kl':3.0e-4,'sc':3.0e-4}), #should lower threshold - only 136 matches with KL
                        'stb3':('20170413',{'kl':7.0e-6,'sc':3.5e-5})  #chose these thresholds to get ~top 400 sites
                        #'sfp1': ##('20170413',{'kl':3.0e-5,'sc':3.0e-5}),
                        #'rsc30': ##('20170413',{'kl':3.0e-5,'sc':3.0e-5}),
                        #'YPR196W': ##('20170413',{'kl':3.0e-5,'sc':1.0e-5}),
                        }
species= ['kl','sc']
#species_dict = {''}

#each motif has the following parameters in a tuple
# index by name: e.g. dot6
# date:  date fimo was run
# thresholds: dictionary with threshold for each species

kl_sc_PKA_data_motif_hits = kl_sc_PKA_data.copy()

#should break out the part that builds the new database separately so that it can be saved and ported elsewehre. 
for motif in motif_parameters_dict.keys(): 
    print(motif)
    # For each species add a column to the data frame to represent the hits.
    for spec in species: 
        # Load Fimo data
        date = motif_parameters_dict[motif][0]
        hits = pd.read_table(data_processing_dir + spec + "_promoters/fimo_output/fimo_" + date + "_"+ spec +"_" + motif + ".txt")
        
        #filter hits
        thresh = motif_parameters_dict[motif][1][spec]
        # # # filter step - could do it by Pvalue or exact motif 
        hits_filtered = hits[hits['p-value']<thresh]
        # #jso_prom_hits_kl_orthologs_exact = jso_prom_hits_kl_orthologs[jso_prom_hits_kl_orthologs['matched sequence'].isin(['AGGGG'])]
        if motif=='stb3':
            hitcount = Counter(hits_filtered['sequence_name'])# counts the elements' frequency  #maybe different versions but stb3 has sequence_name
        else:
            hitcount = Counter(hits_filtered['sequence name'])# counts the elements' frequency  #maybe different versions but stb3 has sequence_name
        hits_list = [hitcount[gene] for gene in kl_sc_PKA_data_motif_hits[spec + '_genename']]
        #add data back to dataframe
        kl_sc_PKA_data_motif_hits[spec + "_" + motif + "_hits"] = hits_list
    
#Save kl_sc_PKA_data_motif_hits 
#kl_sc_PKA_data_motif_hits.to_pickle(base_dir + "\expression_data\kl_PKA_as_20160824\kl_sc_PKA_data_motif_hits")
kl_sc_PKA_data_motif_hits

stb3
dot6
rap1
msn2


,N SCer Orthologs,kl_genename,sc_genename,PKA(AS)+1NMPP1_KL,padj_KL,PKA(AS)+1NMPP1_SC,padj_SC,SC_common_name,kl_stb3_hits,sc_stb3_hits,kl_dot6_hits,sc_dot6_hits,kl_rap1_hits,sc_rap1_hits,kl_msn2_hits,sc_msn2_hits
0,1,KLLA0E08273g,YIL093C,-3.214403,1.019397e-07,-0.188637,7.437343e-01,RSM25,0,2,0,1,0,0,0,0
1,1,KLLA0E02179g,YNL323W,-0.756212,1.901750e-01,1.963392,1.626160e-03,LEM3,0,0,0,0,0,0,0,0
2,1,KLLA0F12056g,YKR076W,4.035633,3.148716e-25,3.899617,2.145520e-32,ECM4,0,0,0,0,0,0,0,2
3,1,KLLA0E19691g,YDL174C,0.452887,6.663246e-01,1.146449,2.666859e-05,DLD1,0,0,0,0,0,0,0,0
4,1,KLLA0F20273g,YOR163W,0.431637,4.496500e-01,1.839392,1.089112e-09,DDP1,0,0,0,0,0,0,0,1
5,1,KLLA0D06127g,YOR246C,-0.459362,3.920201e-01,-1.911512,2.701625e-09,ENV9,0,1,0,0,0,0,0,0
6,1,KLLA0E07063g,YOL137W,-0.340423,7.344797e-01,-0.583991,1.226855e-01,BSC6,0,0,0,0,0,0,1,1
7,1,KLLA0F04499g,YNL067W,-4.451254,7.609392e-22,-1.909883,1.313254e-09,RPL9B,0,1,0,0,0,0,1,1
8,1,KLLA0C15257g,YDR237W,-2.465211,4.810274e-10,-0.067914,9.038782e-01,MRPL7,0,0,1,0,0,0,0,0
9,1,KLLA0E08867g,YNL015W,2.711635,6.669436e-15,3.066771,7.916985e-17,PBI2,0,0,0,0,0,0,1,1


In [37]:
kl_sc_PKA_data_motif_hits.columns
genes_of_interest = ['MRPL13','MSS51','NAM9','QRI3','EAF7']
columns_to_view = ['SC_common_name', 'sc_genename', 'kl_genename','N SCer Orthologs',  'PKA(AS)+1NMPP1_KL',
       'padj_KL', 'PKA(AS)+1NMPP1_SC', 'padj_SC', 
       'kl_stb3_hits', 'sc_stb3_hits']
kl_sc_PKA_data_motif_hits[kl_sc_PKA_data_motif_hits['SC_common_name'].isin(genes_of_interest)].loc[:,columns_to_view]

,SC_common_name,sc_genename,kl_genename,N SCer Orthologs,PKA(AS)+1NMPP1_KL,padj_KL,PKA(AS)+1NMPP1_SC,padj_SC,kl_stb3_hits,sc_stb3_hits
427,MSS51,YLR203C,KLLA0D12430g,1,-2.794425,1.469426e-07,0.819045,0.008024,2,0
1052,NAM9,YNL137C,KLLA0A11660g,1,-0.902406,5.147526e-01,0.517192,0.186289,2,0
1610,MRPL13,YKR006C,KLLA0A02915g,1,-2.541253,5.905259e-05,0.571359,0.132822,2,0
2598,EAF7,YNL136W,KLLA0A11682g,1,-0.668151,2.026145e-01,-0.002766,0.995828,2,0


In [11]:
motif_parameters_dict['stb3'][1]


('20170413', {'kl': 7e-06, 'sc': 3.5e-05})

In [23]:
#Make plot superimposing number of binding sites with in each species on fold change for various transcription factors

motifs = ['msn2','rap1','dot6','stb3']

for jj, motif in enumerate(motifs): 
    print(motif)
    hover_text = []
    for index, row in kl_sc_PKA_data_motif_hits.iterrows():
        hover_text.append(('{SC_common_name}<br>' +
                          'KL '+ motif + ' sites: {kl_motif}<br>'+
                          'SC ' + motif + ' sites: {sc_motif}<br>').format(SC_common_name=row['SC_common_name'],
                                                kl_motif=row['kl_' + motif + '_hits'],
                                                sc_motif=row['sc_' + motif + '_hits']))

    kl_sc_PKA_data_motif_hits['hover_text_' + motif] = hover_text



    data = [
        {
            'x': kl_sc_PKA_data_motif_hits['PKA(AS)+1NMPP1_KL'],
            'y': kl_sc_PKA_data_motif_hits['PKA(AS)+1NMPP1_SC'],
            'text': kl_sc_PKA_data_motif_hits['hover_text_' + motif],
            'mode': 'markers',
            'marker': {
                'color': kl_sc_PKA_data_motif_hits['kl_' + motif + '_hits'],
                'size': kl_sc_PKA_data_motif_hits['sc_' + motif + '_hits']+1,
                'sizeref': 0.2,
                'showscale': True
            }
        }
    ]



    url = py.plot(data, filename= 'bubble_plot_' + motif + '20170823data')





msn2
rap1
dot6
stb3
